<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/06_요약.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  목차
* [Chapter 0 개요](#chapter0)
* [Chapter 1 CNN/DailyMail 데이터셋](#chapter1)

## Chapter 0 개요 <a class="anchor" id="chapter0"></a>
1. 문서의 요약은 긴 단락을 이해하고, 관련 내용을 추론하고, 원본 텍스트의 주제를 통합해 유창한 텍스트를 생성하는 능력이 필요한다.

2. 기사와 법률 계약서의 요약하는 방법은 매우 다르기 때문에 도메인에 맞는 일반화가 필요하다.

3. 사전 훈련된 트랜스포머를 사용해 몬수럴 요약하는 방법을 살펴본다.

4. 인코더-디코더 모델을 만들어 여러 사람이 주고받은 언어를 간결하게 요약해본다.

## Chapter 1 CNN/DailyMail 데이터셋 <a class="anchor" id="chapter1"></a>
1. CNN/DailyMail 데이터셋은 뉴스 기사와 해당 기사의 요약문으로 구성된 대규모 데이터셋이다.
    - 요약문은 기사에 첨부한 글머리 목록의 내용이다.
    - 요약이 본문에서 추출되지 않고 추상적이라는 중요한 특징이 있다.
    - 단순한 발췌가 아니라 새로운 문장으로 구성됐다.

2. 데이터셋은 세 가지 특성이 있다.
    - article: 뉴스 기사 본문
    - highlights: 뉴스 기사의 요약문
    - id: 각 샘플의 고유 식별자

In [2]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
print(f"특성: {dataset['train'].column_names}")

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

특성: ['article', 'highlights', 'id']


In [3]:
sample = dataset["train"][1]
print(f"""기사 문자 발췌, 총 길이: {len(sample['article'])}""")
print(sample["article"][:500], "...")
print(f"\n요약 (길이: {len(sample['highlights'])}):")
print(sample["highlights"])

기사 문자 발췌, 총 길이: 4051
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s ...

요약 (길이: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


3. 기사가 요약에 비해 매우 긴 경우도 있다.
    - 트랜스포모 모델의 문맥 크기가 제한적이기 때문에 긴 기사를 처리하는 데 어려움이 있다.
    - 일반적으로 긴 기사에서 중요한 정보를 추출하는 전처리 단계가 필요하다.
    - 가장 간단한 방법은 모델의 문맥의 크기에 맞게 기사를 자르는 것이다.
    - 텍스트의 끝 부분에 중요한 정보가 있다면 사라지겠지만, 이는 모델 구조의 제약으로 생기는 불가피한 손실이다.